# Example based on our README.md
1. Dataset download
2. Pre-training
3. Fine-tuning (TODO)
4. Inference (TODO)
5. Different backbones
6. Visualize (TODO)
7. Extract features (TODO)


### 1. Download the dataset

In [1]:
# using the cli

# download class dataset
# !python -m wejepa.datasets.download --dataset-root ./data --dataset-name tsbpp/fall2025_deeplearning --splits train

# for development, download a small subset
!python -m wejepa.datasets.download --dataset-root ./data --dataset-name tsbpp/fall2025_deeplearning --splits 'train[:10]'

# download cifar100 dataset
!python -m wejepa.datasets.download --dataset-root ./data --dataset-name cifar100

Split 'train[:10]' available under /home/long/code/dl_project1/experiments/data
Split 'train' available under /home/long/code/dl_project1/experiments/data
Split 'test' available under /home/long/code/dl_project1/experiments/data


### 2. Pre-training the model

In [ ]:
# Using the cli

# Clear
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# Train using default cifar100 config + custom ViT backbone
# !python -m wejepa.train.pretrain --print-config     # print only
# !python -m wejepa.train.pretrain                    # train
_
# FIXME: bug when using .arrow files, the file path is not correctly set, workaround is to rename the arrow file
#   cp fall2025_deeplearning-train.arrow tsbpp___fall2025_deeplearning-train.arrow

# print where --config searches for config files
!python -m wejepa.train.pretrain --config hf224_config.json

In [1]:
# programmatically
from wejepa import default_config, launch_pretraining
cfg = default_config()
launch_pretraining(cfg)

/home/long/code/environments/wejepa/lib/python3.12/site-packages/torch/distributed/distributed_c10d.py:4876: UserWarning: barrier(): using the device under current context. You can specify `device_id` in `init_process_group` to mute this warning.
  warnings.warn(  # warn only once
Training:   0%|          | 0/195 [00:00<?, ?it/s]

Model has 4,449,408 trainable parameters.


Training:   0%|          | 0/195 [00:00<?, ?it/s]

Epoch completed.
Epoch completed.
Epoch completed.
Epoch 1 Iter 50/195 | Loss 0.2569 | 657.3 img/s
Epoch 1 Iter 100/195 | Loss 0.2125 | 1157.1 img/s
Epoch 1 Iter 150/195 | Loss 0.2029 | 1548.1 img/s
Epoch completed.
Saved checkpoint to /home/long/code/dl_project1/experiments/outputs/ijepa/ijepa_epoch_0001.pt
Epoch 1/5 | loss=0.1989


Training:   1%|          | 1/195 [00:00<00:37,  5.24it/s]

...

Epoch completed.
Epoch completed.
Epoch completed.
Epoch 3 Iter 50/195 | Loss 0.1390 | 4895.3 img/s
Epoch 3 Iter 100/195 | Loss 0.1351 | 4866.5 img/s
Epoch 3 Iter 150/195 | Loss 0.1291 | 4910.6 img/s
Epoch completed.
Saved checkpoint to /home/long/code/dl_project1/experiments/outputs/ijepa/ijepa_epoch_0003.pt
Epoch 3/5 | loss=0.1237


Training: 100%|██████████| 195/195 [00:10<00:00, 19.06it/s]

Training:   0%|          | 0/195 [00:00<?, ?it/s]

Epoch completed.
Epoch completed.
Epoch completed.
Epoch 4 Iter 50/195 | Loss 0.0977 | 4783.4 img/s
Epoch 4 Iter 100/195 | Loss 0.0953 | 4783.9 img/s
Epoch 4 Iter 150/195 | Loss 0.0922 | 4839.7 img/s
Epoch completed.
Saved checkpoint to /home/long/code/dl_project1/experiments/outputs/ijepa/ijepa_epoch_0004.pt
Epoch 4/5 | loss=0.0902


Training: 100%|██████████| 195/195 [00:10<00:00, 18.83it/s]

Training: 100%|██████████| 195/195 [00:10<00:00, 18.83it/s]


Epoch 5 Iter 50/195 | Loss 0.0791 | 4786.4 img/s
Epoch 5 Iter 100/195 | Loss 0.0789 | 4837.4 img/s
Epoch 5 Iter 150/195 | Loss 0.0781 | 4848.9 img/s
Epoch completed.
Saved checkpoint to /home/long/code/dl_project1/experiments/outputs/ijepa/ijepa_epoch_0005.pt
Epoch 5/5 | loss=0.0775
Epoch completed.
Epoch completed.
Epoch completed.


### 3. Fine tuning the model

In [ ]:
# using the cli FIXME: broken
!python -m wejepa.train.finetune \
    --checkpoint outputs/ijepa/ijepa_epoch_0005.pt \
    --epochs 10 \
    --batch-size 256 \
    --lr 3e-4 \
    --num-classes 100

In [ ]:
# programmatically FIXME: broken
from wejepa.train import FinetuneConfig, train_linear_probe

ft_cfg = FinetuneConfig(
    checkpoint_path="outputs/ijepa/ijepa_epoch_0005.pt",
    epochs=5,
    batch_size=128,
    learning_rate=1e-3,
)
train_linear_probe(ft_cfg)

### 4. Running Inference

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from datasets import load_dataset
from PIL import Image


from wejepa.train import load_backbone_from_checkpoint
from wejepa import default_config

cfg = default_config()
backbone = load_backbone_from_checkpoint("outputs/ijepa/ijepa_epoch_0005.pt", cfg)
backbone.eval()

transform = transforms.Compose([
    transforms.Resize(cfg.data.image_size),
    transforms.ToTensor(),
    transforms.Normalize(cfg.data.normalization_mean, cfg.data.normalization_std),
])

ds = load_dataset(
    "./data/tsbpp___fall2025_deeplearning",
    split="train",
)

label_feature = ds.features["label"] if hasattr(ds, "features") else None
label_names = label_feature.names if label_feature is not None else None
num_classes = len(label_names) if label_names is not None else 100 # default to 100 classes

decoder = LinearProbe(backbone, num_classes)
decoder.load_state_dict(torch.load("outputs/ijepa/linear_probe.pt", map_location="cpu"))
decoder.eval()

# grab an image from the dataset
image = transform(ds[0]["image"]).unsqueeze(0)
print(f"Image shape: {image.shape}")

with torch.no_grad():
    logits = decoder(image)
    probs = torch.softmax(logits,dim=1)
    pred_ind = int(probs.argmax(dim=1).item())

pred_label = label_names[pred_ind] if label_names is not None else str(pred_ind)
top5_inds = probs.topk(5).indices.squeeze(0).tolist()
top5_labels = [label_names[i] if label_names is not None else str(i) for i in top5_inds]
print(f"Predicted label: {pred_label}")
print(f"Top-5 predicted labels: {top5_labels}")

# remove batch dimension and convert to numpy
img_np = image.squeeze(0).permute(1, 2, 0).cpu().numpy()

# undo normalization for display
mean = np.array(cfg.data.normalization_mean)
std = np.array(cfg.data.normalization_std)
img_np = (img_np * std) + mean
img_np = np.clip(img_np, 0, 1)

plt.imshow(img_np)
plt.axis('off')
plt.show()

with torch.no_grad():
    tokens = backbone(image)
    pooled = tokens.mean(dim=1)  # embeddings for downstream heads

# TODO: use the embeddings `pooled` for downstream tasks like classification 
print(f"Extracted embeddings shape: {pooled.shape}")

num_classes = 100  # adjust based on your dataset
classifier = torch.nn.Linear(pooled.size(1), num_classes)
logits = classifier(pooled)
print(f"Logits shape: {logits.shape}")

# display the classified scores
print(f"Classified scores: {logits}")

# assign predicted class
predicted_class = torch.argmax(logits, dim=1)
print(f"Predicted class: {predicted_class.item()}")

### 5. Different Backbones

In [ ]:
import json
import torch
from copy import deepcopy
from pathlib import Path

from wejepa.backbones import adapt_config_for_backbone, available_backbones
from wejepa.config import IJepaConfig
from wejepa import default_config, launch_pretraining, IJEPA_base

print("Registered backbones: ")
for backbone in available_backbones():
    print(f"- {backbone}")

candidates = ["vit_b_16", "swin_t", "convnext_tiny"]
for backbone in candidates:
    print(f"\nPretraining with backbone: {backbone}")

for backbone in available_backbones():
    cfg = adapt_config_for_backbone(default_config(), backbone)
    print(f"\nBackbone: {backbone}")
    print(f"Image size: {cfg.model.img_size} | Patch size: {cfg.model.patch_size}")

    model = IJEPA_base(
        img_size=cfg.model.img_size,
        patch_size=cfg.model.patch_size,
        in_chans=cfg.model.in_chans,
        embed_dim=cfg.model.embed_dim,
        enc_depth=cfg.model.enc_depth,
        pred_depth=cfg.model.pred_depth,
        num_heads=cfg.model.num_heads,
        backbone=cfg.model.classification_backbone,
        pretrained=cfg.model.classification_pretrained,
    )

    print(f"Total trainable params: {model.count_trainable_parameters() / 1e6:.2f}M")
    print(f"Student + predictor params: {model.count_parameters() / 1e6:.2f}M")

    dummy = torch.randn(1, cfg.model.in_chans, cfg.model.img_size, cfg.model.img_size)
    preds, targets = model(dummy)
    print(f"Pred shape: {tuple(preds.shape)} | Target shape: {tuple(targets.shape)}")
    print(json.dumps(cfg.to_dict(), indent=2))

    cfg.hardware.output_dir = f"./outputs/ijepa/{backbone}"
    cfg_path = Path(f"configs/pretrain_{backbone}.json")
    cfg_path.parent.mkdir(parents=True, exist_ok=True)
    cfg_path.write_text(json.dumps(cfg.to_dict(), indent=2))
    print(f"Saved config for {backbone} at {cfg_path}")

    # launch_pretraining(cfg)

### 6. Visualizing Backbone Embeddings

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

from wejepa.backbones import resolve_preprocess_transforms

class HFImageDataset(Dataset):
    def __init__(self, backbone_name: str, split: str = "train[:256]", label_field: str = "label"):
        self.dataset = load_dataset("./data/tsbpp___fall2025_deeplearning", split=split)
        self.transform = resolve_preprocess_transforms(backbone_name)
        # Some datasets may not provide labels; fall back to a single class for visualization.
        self.label_field = label_field if label_field in self.dataset.features else None

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        image = self.transform(sample["image"])
        label = sample[self.label_field] if self.label_field else 0
        return image, label


def build_dataloader(backbone_name: str, batch_size: int = 24, split: str = "train[:192]") -> DataLoader:
    dataset = HFImageDataset(backbone_name=backbone_name, split=split)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

### 7. Extract Features

In [ ]:
import matplotlib.pyplot as plt

from wejepa.analysis.visualization import (
    extract_backbone_features,
    plot_tsne_embeddings,
    run_tsne_projection,
)
from wejepa.backbones import build_backbone

backbone_names = ["vit_b_16", "swin_t", "convnext_tiny"]
tsne_results = {}

for backbone_name in backbone_names:
    print(f"Projecting embeddings for {backbone_name} ...")
    backbone, feature_dim = build_backbone(backbone_name, pretrained=True, freeze_backbone=True)

    # Use a small slice of the dataset to keep visualization quick.
    dataloader = build_dataloader(backbone_name, batch_size=24, split="train[:192]")
    features, labels = extract_backbone_features(backbone, dataloader, max_batches=4)

    embedding = run_tsne_projection(features, perplexity=20.0, random_state=42)
    fig = plot_tsne_embeddings(embedding, labels)
    fig.suptitle(f"{backbone_name} TSNE", y=1.02)
    plt.show()

    tsne_results[backbone_name] = embedding